# Segmentation Metadata Generator

In [10]:
!pip install --upgrade -q 'supervision>0.14' transformers scipy safetensors 'opencv-python-headless<4.6.0' 'opencv-python<4.6.0' --no-dependencies
!pip install --upgrade -q torch torchvision matplotlib numpy timm
# accelerate cause error in transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.35.2 requires regex!=2019.12.17, which is not installed.
transformers 4.35.2 requires tokenizers<0.19,>=0.14, which is not installed.


In [2]:
!mkdir pretrained_checkpoint/
!wget https://huggingface.co/lkeab/hq-sam/resolve/main/sam_hq_vit_b.pth?download=true -O pretrained_checkpoint/sam_hq_vit_b.pth
!wget https://huggingface.co/lkeab/hq-sam/resolve/main/sam_hq_vit_h.pth?download=true -O pretrained_checkpoint/sam_hq_vit_h.pth

--2023-11-29 00:59:28--  https://huggingface.co/lkeab/hq-sam/resolve/main/sam_hq_vit_b.pth?download=true
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.24, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/d9/4e/d94e04992ac4e3ea45fcc1c523974540cdbef274e295d4aaacc69b4f19ee63b7/14a9d662cd6f5a9c2dba6d40ab0058d88d287e4a18fd6fdc6ad5fb1a3fdeaa57?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sam_hq_vit_b.pth%3B+filename%3D%22sam_hq_vit_b.pth%22%3B&Expires=1701507568&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMTUwNzU2OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9kOS80ZS9kOTRlMDQ5OTJhYzRlM2VhNDVmY2MxYzUyMzk3NDU0MGNkYmVmMjc0ZTI5NWQ0YWFhY2M2OWI0ZjE5ZWU2M2I3LzE0YTlkNjYyY2Q2ZjVhOWMyZGJhNmQ0MGFiMDA1OGQ4OGQyODdlNGExOGZkNmZkYzZhZDVmYjFhM2ZkZWF

In [11]:
import torch
import json
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm
import supervision as sv

from segment_anything import sam_model_registry, SamPredictor

SAM_ENCODER_VERSION = "vit_h"
SAM_CHECKPOINT_PATH = "./pretrained_checkpoint/sam_hq_vit_h.pth"

device = torch.device("cpu")
if torch.cuda.is_available():
   device = torch.device("cuda:0")

sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_CHECKPOINT_PATH)
sam.to(device=device)
sam_predictor = SamPredictor(sam)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/home/kourai/haritsah/sam-hq/segment_anything/modeling/tiny_vit_sam.py:662: UserWarning: Overwriting tiny_vit_5m_224 in registry with segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/kourai/haritsah/sam-hq/segment_anything/modeling/tiny_vit_sam.py:662: UserWarning: Overwriting tiny_vit_11m_224 in registry with segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/kourai/haritsah/sam-hq/segment_anything/modeling/tiny_vit_sam.py:662: UserWarning: Overwriting tiny_vit_21m_224 in registry with segment_anything.modeling.tiny_vit_sam.tiny_vit_21m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return regist

<All keys matched successfully>


# Local Dataset

In [13]:
from hoarder.datasets.local_dataset import LocalDataset
from hoarder.utils.conservator_utils import create_box_annotation, create_polygon_annotation, create_point_annotation
from hoarder.datasets.datasets import get_dataset_by_name

rd = get_dataset_by_name("TestSAM", fields=["name"])
ld = LocalDataset.clone(rd, clone_path='train/data/')
ld.pull()
ld.download()

Cloning into 'train/data/TestSAM'...


Branch 'master' set up to track remote branch 'master' from 'origin'.
Your branch is up to date with 'origin/master'.


Reset branch 'master'


True

In [14]:
from hoarder.utils.conservator_utils import get_label_sets, get_label_id_from_name

label_sets = get_label_sets()
valid_labels = []
for label in label_sets:
    if label.name == "LifeguardSegment":
        valid_labels = [l.name for l in label.labels]

print(valid_labels)

['head', 'person', 'person_in_pool', 'person_submerged']


In [15]:
label_id_dict = {l: get_label_id_from_name(l, label_set_name="LifeguardSegment", label_sets=label_sets) for l in valid_labels}
print(label_id_dict)

{'head': 'NzcLpnhoicyH5qxMK', 'person': '9jDpcQjHG8DB6hn95', 'person_in_pool': 'CRSQmSgpH4vHahxP5', 'person_submerged': 'GvbxFvAxZesXcuQBb'}


# Generate Dataset Metadata

In [16]:
import cv2

if False:
    print("Commiting first changes")
    ld.add_local_changes()
    ld.commit(f'Attempting to add annotations to images')
    ld.pull()

image_frames = ld.get_frames()
for img_idx, frame in tqdm(enumerate(image_frames[1:], 1), total=len(image_frames[1:])):
    # ld.pull()
    image_path = ld.get_local_image_path(frame)
    annotations = frame['annotations']
    image_np = cv2.imread(image_path)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    image_shape = image_np.shape
    sam_predictor.set_image(image_np)

    generated_metadata = {
        "boxes": {}, # xywh
        "polygons": {}, # xy,xy
    }

    for idx, ann in enumerate(annotations):
        if "boundingBox" in ann:
            # ld.pull()
            box = ann["boundingBox"]
            x = int(box["x"])
            y = int(box["y"])
            w = int(box["w"])
            h = int(box["h"])

            label = ann["labels"][0]
            if label not in valid_labels:
                continue

            input_box = torch.tensor([[x, y, x+w, y+h]], device=device)
            transformed_box = sam_predictor.transform.apply_boxes_torch(input_box, image_shape[:2])
            masks, _, _ = sam_predictor.predict_torch(
                point_coords=None,
                point_labels=None,
                boxes=transformed_box,
                multimask_output=False,
                hq_token_only=False,
            )
            masks = masks.squeeze().detach().cpu().numpy() # (H, W)
            polygons = sv.mask_to_polygons(masks) # [np.ndarray([N, 2])]
            if len(polygons) > 0:
                polygons = polygons[0].tolist() # [(x,y), (x,y)]
            else:
                continue
            dict_polygons = [{"x": x_, "y": y_} for x_, y_ in polygons]
            polygon_annotation = create_polygon_annotation(dict_polygons, label, label_id_dict[label], as_dict=True)
            ld.add_annotations_to_frames({img_idx: [polygon_annotation]})



  0%|          | 0/299 [00:00<?, ?it/s]

https://conservator.kour.ai/graphql: HTTP Error 502: Bad Gateway
Retrying request after exception: errors='[{'message': 'HTTP Error 502: Bad Gateway', 'exception': <HTTPError 502: 'Bad Gateway'>, 'status': 502, 'headers': <http.client.HTTPMessage object at 0x7f8ec442f040>, 'body': '<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n</body>\r\n</html>\r\n'}]' operation='query {
  user {
    id
    email
    name
    role
    groups {
      id
      name
    }
    isRemoved
  }
}'
Retry #1
https://conservator.kour.ai/graphql: HTTP Error 502: Bad Gateway
Retrying request after exception: errors='[{'message': 'HTTP Error 502: Bad Gateway', 'exception': <HTTPError 502: 'Bad Gateway'>, 'status': 502, 'headers': <http.client.HTTPMessage object at 0x7f8ec442cee0>, 'body': '<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n</body>\r\n</html>\r\n'}]' operation='query {
  use

In [17]:
ld.add_local_changes()
ld.commit(f'Added new polygon annotations using sam-hq-h')

dataset.jsonl is valid
frames.jsonl is valid
videos.jsonl is valid
[master 6151c9d] Added new polygon annotations using sam-hq-h
 1 file changed, 283 insertions(+), 283 deletions(-)


0

In [18]:
ld.push_commits()

Branch 'master' set up to track remote branch 'master' from 'origin'.
Your branch is up to date with 'origin/master'.


From https://conservator.kour.ai/git/dataset_qkNEgC3rmy29xvfAo
   53d6c17..ddd2a22  master     -> origin/master
Reset branch 'master'


In [19]:
ld.pull()

Branch 'master' set up to track remote branch 'master' from 'origin'.
Your branch is up to date with 'origin/master'.


Reset branch 'master'


0